In [1]:
import os
import sys
WORKING_DIR = os.path.dirname(os.path.abspath('./'))
print(WORKING_DIR)
sys.path.append(WORKING_DIR)

/Users/lorenz/Repos/extractify/backend


In [ ]:
import pandas as pd

df = pd.read_csv("../dataset/mantis_json_dataset_with_intents/mantis_utterance_category.csv")

In [2]:
import json

from entity_linking import EntityLinking
from foundation_models.chat_openai import AIModelType
from schema import topic_schema
current_model = AIModelType.MISTRAL_SMALL


async def link_entities(message: str):
    # generate function call schema
    schema = topic_schema
    
    # gemerate function calling schema
    llm_module = EntityLinking(schema=schema,model=current_model)
    
    print("current_model:", current_model)
    
    if current_model == AIModelType.MISTRAL_LARGE or current_model == AIModelType.MISTRAL_MIXTRAL_8x22B or current_model == AIModelType.MISTRAL_SMALL:
        filter_generator_output = llm_module.generate_sync(conversation=message)
        return filter_generator_output
    if current_model == AIModelType.GPT4_O or current_model == AIModelType.GPT4_TURBO or current_model == AIModelType.GPT4_O_MINI or current_model == AIModelType.GPT3 or current_model == AIModelType.GOOGLE_GEMINI_PRO or current_model == AIModelType.CLAUDE_OPUS or current_model == AIModelType.CLAUDE_SONNET:
        filter_generator_output = await llm_module.generate_response_generic(conversation=message)
    else:
        filter_generator_output = await llm_module.generate_async(conversation=message)
        
    print("filter_generator_output:", filter_generator_output)
    
    return filter_generator_output
        
    
# test the function
message = "What is the difference when speaking about 'LOB' and 'binary' data? Is it the same, in terms of binary data being stored in a separate filegroup, or are there differences?"
response = await link_entities(message)
print("response:", response)

# save to json file
with open('./temp.json', 'w') as f:
    json.dump(response, f, indent=4, ensure_ascii=False)

current_model: AIModelType.GOOGLE_GEMINI_PRO
raw response candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "entity_linking"
        args {
          fields {
            key: "category"
            value {
              string_value: "dba"
            }
          }
        }
      }
    }
  }
  avg_logprobs: -0.00019140884978696704
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0966796875
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.115722656
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.486328125
    severity: HARM_SEVERITY_LOW
    severity_score: 0.38671875
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.122558594
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0673828125
  }
  safety_rat

In [ ]:
# import pandas as pd
# import json
# df = pd.read_json(f"prediction_mantis_{current_model}_v1.json")

In [ ]:
# iterate over rows with iterrows()
import asyncio
import json

tasks = []

for index, row in df.iterrows():
    # predicted_entities = row[2]

    # if predicted_entities is not None and type(predicted_entities) == str:
    #     print("Already predicted entities")
    #     continue

    print(row)
    query = row[0]
    actual_category = row[1]
    print(actual_category)
    print(query)

    async def recognize_and_return_index(
        index, message: str
    ):
        recognized_filters = (
            await link_entities(
                message=message,
            )
        )
        
        
        predicted_category = recognized_filters.get("category")
        
        if predicted_category is None:
            predicted_category = "None"
        
        print(predicted_category)

        df.at[index, "Prediction"] = predicted_category
        return index

    print(index)
    
    try:
        await recognize_and_return_index(
        index=index,
        message=query,
    )
    except Exception as e:
        print(e)
        continue

    # wait to avoid rate limiting
    await asyncio.sleep(5)

 
    # write to csv
    df.to_json(f"prediction_mantis_{current_model}_v1.json", index=False, orient="records")


print(len(tasks))

In [ ]:

current_model = AIModelType.GOOGLE_GEMINI_PRO

# import pandas as pd
import json
df = pd.read_json(f"./mantis-category-prediction/prediction_mantis_{current_model}_v1.json")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def classification_metrics(true_labels, predicted_labels):
    """
    Calculate and print classification metrics.

    Parameters:
    - true_labels: List of true labels
    - predicted_labels: List of predicted labels

    Returns:
    - metrics: Dictionary containing accuracy, precision, recall, and F1-score
    """
    metrics = {
        'accuracy': accuracy_score(true_labels, predicted_labels),
        'precision': precision_score(true_labels, predicted_labels, average='weighted'),
        'recall': recall_score(true_labels, predicted_labels, average='weighted'),
        'f1_score': f1_score(true_labels, predicted_labels, average='weighted')
    }

    print(f"Accuracy: {metrics['accuracy']:.2f}")
    print(f"Precision: {metrics['precision']:.2f}")
    print(f"Recall: {metrics['recall']:.2f}")
    print(f"F1 Score: {metrics['f1_score']:.2f}")

    return metrics

# Example usage
true_labels = df["category"].tolist()
predicted_labels = df["Prediction"].tolist()

classification_metrics(true_labels, predicted_labels)

We have lots of false negatives, as the model does not have all filter ids in schema for Function Call Model. We use custom filter ids for the experiment, to give them a fair chance.